In [37]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [57]:
import camb

In [58]:
z,_=loadtxt("camb_rpc/cosmomc/camb/xefid.dat").T

dl=20
w=hstack([ones(len(z)-dl),(cos(pi*arange(dl)/(dl-1))+1)/2])
modes=loadtxt("camb_rpc/cosmomc/camb/xepcs.dat")[1:]*w

cp=camb.set_params(As=1e-9)
r=camb.get_results(cp)
fidxe=cp.Reion.get_xe(1/(1+z))

In [59]:
def get_f(ps):
    
    par = ps.values()
    nobs = 3
    npar = len(par)
    
    def run(par):

        cp=camb.set_params(lmax=5000,**{k:v for k,v in zip(ps,par) if 'xe_m' not in k})
        cp.k_eta_max_scalar=10000
        cp.DoLensing=True
        cp.NonLinear=2
        xe=fidxe.copy()
        for i in range(95):
            try:
                j=list(ps).index('xe_m_%i'%i)            
            except ValueError:
                pass
            else:
                xe+=par[j]*modes[i]
                cp.Reion.set_xe(1/(1+z),xe)
        r=camb.get_results(cp)

        ucls = r.get_unlensed_scalar_cls(lmax)
        lcls = r.get_lens_potential_cls(lmax)

        ell = arange(lmin,lmax)
        scale = (2.73e6)**2*2*pi/ell/(ell+1) 

        tt,ee,te = scale*ucls[lmin:lmax,[0,1,3]].T
        td,dd    = scale*lcls[lmin:lmax,[1,0]].T

        return ell, tt, ee, dd, te, td 


    def cov_obs(par, Tels):
        Cov  = zeros((lmax, nobs, nobs)) 

        fsky, noise_T, noise_P, beam = Tels
        ell, tt, ee, dd, te, td = run(par)

        for i in range(len(ell)):
            l = ell[i]
            Blm2 = exp(l*(l+1)*beam**2/(8*log(2)))        

            Cov[i][0][0]  = tt[i] + noise_T*Blm2
            Cov[i][1][1]  = ee[i] + noise_P*Blm2
            Cov[i][2][2]  = dd[i] + noise_dd[i] 
            Cov[i][0][1]  = te[i]
            Cov[i][1][0]  = te[i]
            Cov[i][0][2]  = td[i]
            Cov[i][2][0]  = td[i]
        return ell, Cov

    def finite_diff(par, Tels):
        Cov_par = zeros((lmax, npar, nobs, nobs))

        for i in range(npar):
            par_l    = [x for x in par]
            par_r    = [x for x in par]

#             if 'xe_m' in list(ps)[i]:
            if par[i]==0:
                delta = 0.03
            else:
                delta = par[i]*0.03

            par_l[i] = par[i]-delta
            par_r[i] = par[i]+delta
            print i, par[i], par_l[i], par_r[i]

            ell, Cov_l = cov_obs(par_l, Tels)
            ell, Cov_r = cov_obs(par_r, Tels)

            Cov_par[:,i,:,:] = (Cov_r-Cov_l)/(2*delta)

        return  Cov_par


    def fisher(par, Tels):
        alpha     = zeros((npar, npar))
        alpha_l   = zeros(lmax)
        fsky      = Tels[0]

        ell, Cov  = cov_obs(par, Tels)
        Cov_par   = finite_diff(par, Tels)

        for i in range(npar):
            for j in range(npar):
                for l in range(len(ell)):
                    Hess = inv(Cov[l,:,:])
                    matr = dot(dot(Hess, Cov_par[l,i,:,:]), dot(Hess, Cov_par[l,j,:,:]))
                    alpha_l[l] = (2*ell[l]+1)/2.*fsky*trace(matr)
                alpha[i][j] = sum(alpha_l)

        return alpha




    #               fsky    noise_T            noise_P            FWHM
    CMB_S4 = array([0.5, (1.5/60./57.4)**2, 2*(1.5/60./57.4)**2, 1./60./57.4])

    lmin, lmax = 20, 3000

    noise_dd = (2.73e6**2)*loadtxt('noise_dd.dat')
    

    alpha = fisher(par, CMB_S4)
    if 'tau' in ps:
        i=list(ps).index('tau'); alpha[i][i] += 1/0.01**2 #tau prior
    return alpha

In [54]:
ps_tau = {
    'As':2.215e-9,
    'ns':0.9619,
    'tau':0.095,
    'H0':67,
    'ombh2':0.022068,
    'omch2':0.1209,
    'mnu':0.085,
    'nrun':0,
}

ps_reio=ps_tau.copy()
ps_reio.pop('tau')
for i in range(5): ps_reio['xe_m_%i'%i]=0

In [55]:
fcmb_tau = get_f(ps_tau)
fcmb_reio = get_f(ps_reio)

0 0.095 0.09215 0.09785
1 2.215e-09 2.14855e-09 2.28145e-09
2 0.1209 0.117273 0.124527
3 0 -0.03 0.03
4 67 64.99 69.01
5 0.9619 0.933043 0.990757
6 0.022068 0.02140596 0.02273004
7 0.085 0.08245 0.08755


In [43]:
fdesi=loadtxt("s4/F_DESI.dat")
fdesi[3]/=100
fdesi[:,3]/=100

a=identity(7)
a[4:6,4:6]=inv(array([[1,-1],
                      [0,1]]))
fdesi=dot(a.T,dot(fdesi,a))

In [44]:
def add_desi(f,ps):
    f=f.copy()
    idesi=[3,6,5,4]
    icmb=[list(ps).index(k) for k in ['H0','mnu','omch2','ombh2']]
    f[ix_(icmb,icmb)]+=fdesi[ix_(idesi,idesi)]
    return f

In [45]:
def get_sigma_p(f,ps,p):
    i=list(ps).index(p)
    return sqrt(inv(f)[i,i])

def get_sigma_mnu(f,ps): 
    return get_sigma_p(f,ps,'mnu')

In [56]:
get_sigma_mnu(fcmb_tau,ps_tau), get_sigma_mnu(fcmb_reio,ps_reio)

(0.051822790341627546, 13.244134720797211)

In [51]:
get_sigma_mnu(fcmb_tau,ps_tau), get_sigma_mnu(fcmb_reio,ps_reio)

(0.048817741936286983, 0.078595069495662959)

In [47]:
get_sigma_mnu(fcmb_tau,ps_tau), get_sigma_mnu(fcmb_reio,ps_reio)

(0.056186432534400116, 0.078595069495662959)

In [48]:
get_sigma_mnu(add_desi(fcmb_tau,ps_tau),ps_tau), get_sigma_mnu(add_desi(fcmb_reio,ps_reio),ps_reio)

(0.032639634907481778, 0.045990935010054322)